In [50]:
from firecrawl import FirecrawlApp
import requests 
import scrapy

In [39]:
# !pip install docling



In [ ]:
from utils.aws import s3
import os 
from dotenv import load_dotenv
import requests
import json
from boto3.s3.transfer import TransferConfig
from docling.datamodel.pipeline_options import PipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

load_dotenv()


pipeline_options = PipelineOptions()
pipeline_options.generate_picture_images = True  # Enable picture image generation
pipeline_options.images_scale = 1.0  # Adjust image resolution scale if needed

# Initialize the DocumentConverter with pipeline options
converter = DocumentConverter(
    format_options={
        "PDF": PdfFormatOption(pipeline_options=pipeline_options)
    }
)


# def store_data_to_s3():
headers = {
    'User-Agent': 'MIT  bigdata@gmail.com',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Host': 'www.sec.gov',
    'Connection': 'keep-alive'
    }

bucket_name = os.getenv('BUCKET_NAME')
region = os.getenv('REGION')
s3_client = s3.get_s3_client()
json_link_key = "metadata/metadata.json"

s3_url =f"https://{bucket_name}.s3.{region}.amazonaws.com/{json_link_key}"

response = requests.get(s3_url)
data =  response.json()

year = '2025'
qtr = f'Fourth Quarter {year}'

file_link = data[year][qtr]

source = "https://rag-pipeline-data.s3.us-east-2.amazonaws.com/p.pdf" # document per local path or URL
# converter = DocumentConverter()
result = converter.convert(source)
md_content = result.document.export_to_markdown()


md_file = f"nvidia_{year}_{qtr}"
with open(f"output/{md_file}", "w", encoding="utf-8") as file:
    file.write(md_content)

output_img = "images"
os.makedirs(output_img, exist_ok=True)

for i, picture in enumerate(result.document.pictures):
    image_data = picture.get_image()
    if image_data:  # Ensure image exists
        image_path = os.path.join(output_img, f"image_{i + 1}.png")
        with open(image_path, "wb") as img_file:
            img_file.write(image_data)
        print(f"Saved image {i + 1} to {image_path}")


# with requests.get(file_link, stream = True, headers=headers) as file:

    

ModuleNotFoundError: No module named 'docling_core.transforms.converter'

{'2025': {'Fourth Quarter 2025': 'https://s201.q4cdn.com/141608511/files/doc_financials/2025/q4/177440d5-3b32-4185-8cc8-95500a9dc783.pdf', 'Third Quarter 2025': 'https://s201.q4cdn.com/141608511/files/doc_financials/2025/q3/ed2a395c-5e9b-4411-8b4a-a718d192155a.pdf', 'Second Quarter 2025': 'https://s201.q4cdn.com/141608511/files/doc_financials/2025/q2/78501ce3-7816-4c4d-8688-53dd140df456.pdf', 'First Quarter 2025': 'https://s201.q4cdn.com/141608511/files/doc_financials/2025/q1/NVIDIA-10Q-20242905.pdf'}}


In [ ]:
from utils.aws import s3
import os 
from dotenv import load_dotenv
import requests
import json
from boto3.s3.transfer import TransferConfig

from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption


load_dotenv()

# IMAGE_RESOLUTION_SCALE = 2.0

pipeline_options = PdfPipelineOptions()
pipeline_options.generate_picture_images = True
pipeline_options.images_scale = 1.0

converter = DocumentConverter(
    format_options={
        "PDF": PdfFormatOption(pipeline_options=pipeline_options)
    }
)


# def store_data_to_s3():
headers = {
    'User-Agent': 'MIT  bigdata@gmail.com',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Host': 'www.sec.gov',
    'Connection': 'keep-alive'
    }

bucket_name = os.getenv('BUCKET_NAME')
region = os.getenv('REGION')
s3_client = s3.get_s3_client()

json_link_key = "metadata/metadata.json"
s3_url =f"https://{bucket_name}.s3.{region}.amazonaws.com/{json_link_key}"

response = requests.get(s3_url)
data =  response.json()

year = '2025'
qtr = f'Fourth Quarter {year}'
file_link = data[year][qtr]

#convert to md
source = "https://rag-pipeline-data.s3.us-east-2.amazonaws.com/p.pdf" # document per local path or URL
result = converter.convert(source)
md_content = result.document.export_to_markdown()


md_file = f"nvidia_{year}"
# output_md =f"{md_file}"
# os.makedirs(output_md, exist_ok=True)

# with open(f"{output_md}/content.md", "w", encoding="utf-8") as file:
#     file.write(md_content)


output_img = "images"
os.makedirs(output_img, exist_ok=True)

for i, picture in enumerate(result.document.pictures):
    image_data = picture.get_image(result.document)
    if image_data:  # Ensure image exists
        print("img found")
        local_image_path = os.path.join(output_img, f"image_{i + 1}.png")
        s3_key = f"images/nvidia_{qtr}_image_{i + 1}.png"
        
        # with open(local_image_path, "wb") as img_file:
        #     img_file.write(image_data)

        # s3_client.upload_file(local_image_path, Bucket=bucket_name, Key=s3_key)
        s3_img_url = f"https://{bucket_name}.s3.{region}.amazonaws.com/{s3_key}"

        md_content = md_content.replace(f"![Image {i + 1}](placeholder_url)", f"![Image {i + 1}]({s3_img_url})")
    else:
        print("no img found")

updated_md_file = f"output/updated_{md_file}.md"
os.makedirs("output", exist_ok=True)
with open(updated_md_file, "w", encoding = "utf-8") as file:
    file.write(md_content)

# with requests.get(file_link, stream = True, headers=headers) as file:

    

no img found
no img found
no img found
no img found
no img found
no img found
no img found


In [78]:
import re
t =  "Fourth Quarter 2025"

# s.replace(" ","_")
r =  re.sub(r"\s+", "_", "Fourth Quarter 2025")

In [92]:
# s3_client.list_buckets()


In [116]:
json_link_key = "metadata/metadata.json"


s3_url =f"https://{bucket_name}.s3.{region}.amazonaws.com/{json_link_key}"
s3_url

'https://rag-pipeline-data.s3.us-east-2.amazonaws.com/metadata/metadata.json'

In [163]:
daa = requests.get(s3_url)
json_data = json.loads(daa.content)
len(json_data)
json_data
for key,value in json_data.items():
    for qtr, link in json_data[key].items():
        print(link)




    


https://s201.q4cdn.com/141608511/files/doc_financials/2022/q4/ca04d49f-caab-436f-9e6f-e8493c8e8515.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2022/q3/420f1223-6131-4b97-b09d-3e9935257ed2.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2022/q2/62e21663-af56-4c3a-b0d1-98bfdaa5d4a6.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2022/q1/NVDA-2022-Q1-10Q.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2023/q4/4e9abe7b-fdc7-4cd2-8487-dc3a99f30e98.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2023/q3/cd532449-4dc3-404a-a999-25e7b2e8c246.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2023/Q2/19426b68-6120-44a3-9032-bb629ef2b3d9.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2023/q1/8b8d4537-83d8-4c94-b065-e7a2cd34da49.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2024/q4/1cbe8fe7-e08a-46e3-8dcc-b429fc06c1a4.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2024/q3/NVIDIA-10Q.pdf
https://s201.q4c

In [2]:
# json_data["2025"]
import re

In [5]:
qtr = "Third Quarter 2025"
match = re.search(r"(first|second|third|fourth)", qtr, re.IGNORECASE)
if match:
    quarter_map = {"first": "1", "second": "2", "third": "3", "fourth": "4"}
    q = quarter_map[match.group(1).lower()]

q

'3'

In [165]:

for year,qtrs in json_data.items():
        for qtr, link in json_data[year].items():
            file_link = json_data[year][qtr]
            print(file_link)

https://s201.q4cdn.com/141608511/files/doc_financials/2022/q4/ca04d49f-caab-436f-9e6f-e8493c8e8515.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2022/q3/420f1223-6131-4b97-b09d-3e9935257ed2.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2022/q2/62e21663-af56-4c3a-b0d1-98bfdaa5d4a6.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2022/q1/NVDA-2022-Q1-10Q.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2023/q4/4e9abe7b-fdc7-4cd2-8487-dc3a99f30e98.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2023/q3/cd532449-4dc3-404a-a999-25e7b2e8c246.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2023/Q2/19426b68-6120-44a3-9032-bb629ef2b3d9.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2023/q1/8b8d4537-83d8-4c94-b065-e7a2cd34da49.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2024/q4/1cbe8fe7-e08a-46e3-8dcc-b429fc06c1a4.pdf
https://s201.q4cdn.com/141608511/files/doc_financials/2024/q3/NVIDIA-10Q.pdf
https://s201.q4c

In [169]:
api_key = os.getenv('MISTRAL_API_KEY')

print(api_key)

None


<class 'mistralai.models.ocrresponse.OCRResponse'>


In [ ]:
from io import BytesIO

data_buffer = BytesIO(ocr_response)

with open("output.md", "wb") as file:
    file.write(data_buffer)

TypeError: a bytes-like object is required, not 'OCRResponse'

In [176]:

def metadataLinks():
    bucket_name = os.getenv('BUCKET_NAME')
    region = os.getenv('REGION')
    s3_client = s3.get_s3_client()

    json_link_key = "metadata/metadata.json"
    s3_url =f"https://{bucket_name}.s3.{region}.amazonaws.com/{json_link_key}"

    response = requests.get(s3_url)
    json_data = json.loads(response.content)

    return json_data

In [177]:
print(metadataLinks())

{'2022': {'Fourth_Quarter_2022': 'https://s201.q4cdn.com/141608511/files/doc_financials/2022/q4/ca04d49f-caab-436f-9e6f-e8493c8e8515.pdf', 'Third_Quarter_2022': 'https://s201.q4cdn.com/141608511/files/doc_financials/2022/q3/420f1223-6131-4b97-b09d-3e9935257ed2.pdf', 'Second_Quarter_2022': 'https://s201.q4cdn.com/141608511/files/doc_financials/2022/q2/62e21663-af56-4c3a-b0d1-98bfdaa5d4a6.pdf', 'First_Quarter_2022': 'https://s201.q4cdn.com/141608511/files/doc_financials/2022/q1/NVDA-2022-Q1-10Q.pdf'}, '2023': {'Fourth_Quarter_2023': 'https://s201.q4cdn.com/141608511/files/doc_financials/2023/q4/4e9abe7b-fdc7-4cd2-8487-dc3a99f30e98.pdf', 'Third_Quarter_2023': 'https://s201.q4cdn.com/141608511/files/doc_financials/2023/q3/cd532449-4dc3-404a-a999-25e7b2e8c246.pdf', 'Second_Quarter_2023': 'https://s201.q4cdn.com/141608511/files/doc_financials/2023/Q2/19426b68-6120-44a3-9032-bb629ef2b3d9.pdf', 'First_Quarter_2023': 'https://s201.q4cdn.com/141608511/files/doc_financials/2023/q1/8b8d4537-83d8-4

In [179]:
from mistralai import Mistral
import os 
from mistralai.models import OCRResponse
from IPython.display import Markdown, display
from dotenv import load_dotenv
# from utils.aws.s3 import metadataLinks 
load_dotenv()


api_key = os.getenv('MISTRAL_API_KEY')
client = Mistral(api_key=api_key)
# text_model = "mistral-small-latest"
# ocr_model = "mistral-ocr-latest"

url = "https://s201.q4cdn.com/141608511/files/doc_financials/2022/q4/ca04d49f-caab-436f-9e6f-e8493c8e8515.pdf"


def ocr_response(url):
    ocr_response = client.ocr.process(
        model="mistral-ocr-latest",
        document={
            "type": "document_url",
            "document_url": url
        },
        include_image_base64=True
    )
    return ocr_response


# Display combined markdowns and images
# display(Markdown(get_combined_markdown(pdf_response)))

# get_combined_markdown(response)
# print(response)

# if __name__ == "__main__":
#     data = metadataLinks()
#     for year,qtrs in data.items():
#             for qtr, link in data[year].items():
#                 file_link = data[year][qtr]

#                 response = ocr_response(file_link)
#                 markdown_file = get_combined_markdown(response)

In [196]:

import base64
year = 2022
qtr = f"Fourth_Quarter_{year}"
def replace_images_in_markdown(markdown_str: str, images_dict: dict) -> str:

    for img_name, base64_str in images_dict.items():
        img_data = base64_str.split(';')[1].split(',')[1]
        image_data = base64.b64decode(img_data)
        s3_image_key = f"{year}/{qtr}/mistral/images/{img_name}"  

        s3_client.put_object(
            Bucket=bucket_name,
            Key=s3_image_key,
            Body=image_data,
            ContentType="image/png"
        )
        s3_url = f"https://{bucket_name}.s3.{region}.amazonaws.com/{s3_image_key}"
        markdown_str = markdown_str.replace(f"![{img_name}]({img_name})", f"![{img_name}]({s3_url})")
 
    return markdown_str

def get_combined_markdown(ocr_response: OCRResponse) -> str:
    markdowns: list[str] = []
    # Extract images from page
    for page in ocr_response.pages:
        image_data = {}
        for img in page.images:
            image_data[img.id] = img.image_base64
        # Replace image placeholders with actual images
        markdowns.append(replace_images_in_markdown(page.markdown, image_data))
    return "\n\n".join(markdowns)



In [197]:

response = ocr_response(url)
markdown_file = get_combined_markdown(ocr_response=response)

In [198]:
markdown_file = get_combined_markdown(response)

s3_key = f"{year}/{qtr}/mistral/{markdown_file}.md"

In [ ]:
markdown_file


'# UNITED STATES <br> SECURITIES AND EXCHANGE COMMISSION <br> Washington, D.C. 20549 \n\nFORM 10-K\n20 ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended January 30, 2022\nOR\n$\\square$ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nCommission file number: 0-23985\n![img-0.jpeg](https://rag-pipeline-data.s3.us-east-2.amazonaws.com/2022/Fourth_Quarter_2022/mistral/images/img-0.jpeg)\n\nIndicate by check mark if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act. Yes $\\boxtimes$ No $\\square$\nIndicate by check mark if the registrant is not required to file reports pursuant to Section 13 or Section 15(d) of the Act. Yes $\\square$ No $\\boxtimes$\nIndicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 during the preceding 12 months (or for such